In [1]:
import requests
import json

# Hardcoded raw GitHub URL (not blob)
raw_url = "https://raw.githubusercontent.com/PeterCullenBurbery/windows-file-name-rules/main/file_names/file-names-002/windows_filename_rules.json"

try:
    response = requests.get(raw_url, timeout=10)
    response.raise_for_status()  # Raises an HTTPError for bad responses
    data = response.json()
except requests.RequestException as e:
    print(f"❌ Failed to load data: {e}")
    data = None

# Show parsed data
if data:
    print(json.dumps(data, indent=2))

{
  "table_of_characters": [
    {
      "id": "space",
      "char": " ",
      "web": true,
      "file": true
    },
    {
      "id": "forward_slash",
      "char": "/",
      "web": true,
      "file": false
    },
    {
      "id": "backslash",
      "char": "\\",
      "web": true,
      "file": false
    },
    {
      "id": "colon",
      "char": ":",
      "web": true,
      "file": false
    },
    {
      "id": "asterisk",
      "char": "*",
      "web": true,
      "file": false
    },
    {
      "id": "question_mark",
      "char": "?",
      "web": true,
      "file": false
    },
    {
      "id": "less_than",
      "char": "<",
      "web": true,
      "file": false
    },
    {
      "id": "greater_than",
      "char": ">",
      "web": true,
      "file": false
    },
    {
      "id": "pipe",
      "char": "|",
      "web": true,
      "file": false
    },
    {
      "id": "double_quote",
      "char": "\"",
      "web": true,
      "file": false
    }
  ],
  "tab

In [2]:
print(len(data['reserved_names']))

22


In [4]:
import json
import time
import requests
from peter_cullen_burbery_python_functions.system_management_functions import convert_blob_to_raw_github_url

def validate_Windows_filename_with_reasons(name: str) -> dict:
    """
    Validates a Windows filename using rules from a GitHub-hosted JSON file.
    Retries up to 100 times using the `requests` library for more robust HTTP handling.
    """
    # GitHub blob URL containing the JSON rules
    blob_url = (
        "https://github.com/PeterCullenBurbery/windows-file-name-rules/blob/main/"
        "file_names/file-names-002/windows_filename_rules.json"
    )

    # Convert to raw content URL
    raw_url = convert_blob_to_raw_github_url(blob_url)

    # Attempt to download the JSON with retry logic
    for attempt in range(1, 101):
        try:
            response = requests.get(raw_url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                break
            else:
                raise requests.HTTPError(f"Status code {response.status_code}")
        except Exception as e:
            if attempt == 100:
                raise RuntimeError(f"❌ Failed to load JSON after 100 attempts: {e}")
            time.sleep(0.5)

    # Build lookup tables
    char_id_to_char = {entry["id"]: entry["char"] for entry in data["table_of_characters"]}
    char_to_id = {
        entry["char"]: entry["id"]
        for entry in data["table_of_characters"]
        if not entry["file"]
    }
    group_id_to_description = {
        entry["id"]: entry["description"] for entry in data["table_of_groups_of_characters"]
    }
    char_id_to_group_id = {
        entry["character_id"]: entry["group_id"]
        for entry in data["table_of_characters_in_groups"]
    }
    reserved_names_dict = {
        entry["name"]: entry["reason"] for entry in data["reserved_names"]
    }

    # Track invalid characters and reasons
    invalids = []

    # Reason 1: Disallowed characters
    for char in name:
        if char in char_to_id:
            char_id = char_to_id[char]
            group_id = char_id_to_group_id.get(char_id)
            reason = group_id_to_description.get(group_id, "Disallowed character")
            invalids.append((char, reason))

    # Reason 2: Ends with space or period
    if name.endswith(" ") or name.endswith("."):
        invalids.append((
            name[-1],
            "File or folder names cannot end with a space or period (.)"
        ))

    # Reason 3: Reserved device names
    base_name = name.split('.')[0].upper()
    if base_name in reserved_names_dict:
        invalids.append((
            base_name,
            reserved_names_dict[base_name]
        ))

    if not invalids:
        return {"valid": True}
    else:
        return {
            "valid": False,
            "problems": [
                {"character": c, "reason": r} for c, r in invalids
            ]
        }

In [5]:
import unittest

class TestWindowsFilenameValidation(unittest.TestCase):

    def test_reserved_names(self):
        # Dynamically load reserved names from the data dictionary
        reserved = [entry["name"] for entry in data["reserved_names"]]

        for name in reserved:
            with self.subTest(name=name):
                result = validate_Windows_filename_with_reasons(name)
                self.assertFalse(result["valid"])
                self.assertTrue(any("Reserved" in p["reason"] for p in result["problems"]))

unittest.main(argv=[''], verbosity=2, exit=False)

test_reserved_names (__main__.TestWindowsFilenameValidation.test_reserved_names) ... ok

----------------------------------------------------------------------
Ran 1 test in 6.042s

OK


In [6]:
import unittest

class TestWindowsFilenameValidation(unittest.TestCase):

    def test_reserved_names(self):
        # Dynamically load reserved names from the data dictionary
        reserved = [entry["name"] for entry in data["reserved_names"]]

        for name in reserved:
            with self.subTest(name=name):
                result = validate_Windows_filename_with_reasons(name)
                print(f"Testing {name}: {result}")  # 👈 This shows the output for each name
                self.assertFalse(result["valid"])
                self.assertTrue(any("Reserved" in p["reason"] for p in result["problems"]))

unittest.main(argv=[''], verbosity=2, exit=False)

test_reserved_names (__main__.TestWindowsFilenameValidation.test_reserved_names) ... 

Testing CON: {'valid': False, 'problems': [{'character': 'CON', 'reason': 'Reserved device name: console'}]}
Testing PRN: {'valid': False, 'problems': [{'character': 'PRN', 'reason': 'Reserved device name: printer'}]}
Testing AUX: {'valid': False, 'problems': [{'character': 'AUX', 'reason': 'Reserved device name: auxiliary device'}]}
Testing NUL: {'valid': False, 'problems': [{'character': 'NUL', 'reason': 'Reserved device name: null device'}]}
Testing COM1: {'valid': False, 'problems': [{'character': 'COM1', 'reason': 'Reserved device name: serial port COM1'}]}
Testing COM2: {'valid': False, 'problems': [{'character': 'COM2', 'reason': 'Reserved device name: serial port COM2'}]}
Testing COM3: {'valid': False, 'problems': [{'character': 'COM3', 'reason': 'Reserved device name: serial port COM3'}]}
Testing COM4: {'valid': False, 'problems': [{'character': 'COM4', 'reason': 'Reserved device name: serial port COM4'}]}
Testing COM5: {'valid': False, 'problems': [{'character': 'COM5', 'reas

ok

----------------------------------------------------------------------
Ran 1 test in 3.628s

OK


Testing LPT8: {'valid': False, 'problems': [{'character': 'LPT8', 'reason': 'Reserved device name: parallel port LPT8'}]}
Testing LPT9: {'valid': False, 'problems': [{'character': 'LPT9', 'reason': 'Reserved device name: parallel port LPT9'}]}


In [7]:
from peter_cullen_burbery_python_functions.system_management_functions import convert_blob_to_raw_github_url

raw_url = convert_blob_to_raw_github_url("https://github.com/PeterCullenBurbery/windows-file-name-rules/blob/main/file_names/file-names-002/windows_filename_rules.json")
print(raw_url)

https://github.com/PeterCullenBurbery/windows-file-name-rules/raw/main/file_names/file-names-002/windows_filename_rules.json


In [9]:
import unittest
import requests
import json
import time

from peter_cullen_burbery_python_functions.system_management_functions import (
    convert_blob_to_raw_github_url,
    validate_Windows_filename_with_reasons
)

class TestWindowsFilenameValidation(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        # Load the JSON rules from GitHub with retry
        blob_url = (
            "https://github.com/PeterCullenBurbery/windows-file-name-rules/blob/main/"
            "file_names/file-names-002/windows_filename_rules.json"
        )
        raw_url = convert_blob_to_raw_github_url(blob_url)

        for attempt in range(1, 101):
            try:
                response = requests.get(raw_url, timeout=10)
                response.raise_for_status()
                cls.data = response.json()
                break
            except Exception as e:
                if attempt == 100:
                    raise RuntimeError(f"❌ Failed to load JSON after 100 attempts: {e}")
                time.sleep(0.5)

    def test_reserved_names(self):
        reserved = [entry["name"] for entry in self.data["reserved_names"]]

        for name in reserved:
            with self.subTest(name=name):
                result = validate_Windows_filename_with_reasons(name)
                print(f"🔍 Testing {name} → {result}")
                self.assertFalse(result["valid"], f"{name} should be invalid")
                self.assertTrue(
                    any("Reserved" in p["reason"] for p in result["problems"]),
                    f"Expected 'Reserved' reason in problems for {name}"
                )

if __name__ == "__main__":
    unittest.main(argv=[""], verbosity=2, exit=False)

test_reserved_names (__main__.TestWindowsFilenameValidation.test_reserved_names) ... 

🔍 Testing CON → {'valid': False, 'problems': [{'character': 'CON', 'reason': 'Reserved device name: console'}]}
🔍 Testing PRN → {'valid': False, 'problems': [{'character': 'PRN', 'reason': 'Reserved device name: printer'}]}
🔍 Testing AUX → {'valid': False, 'problems': [{'character': 'AUX', 'reason': 'Reserved device name: auxiliary device'}]}
🔍 Testing NUL → {'valid': False, 'problems': [{'character': 'NUL', 'reason': 'Reserved device name: null device'}]}
🔍 Testing COM1 → {'valid': False, 'problems': [{'character': 'COM1', 'reason': 'Reserved device name: serial port COM1'}]}
🔍 Testing COM2 → {'valid': False, 'problems': [{'character': 'COM2', 'reason': 'Reserved device name: serial port COM2'}]}
🔍 Testing COM3 → {'valid': False, 'problems': [{'character': 'COM3', 'reason': 'Reserved device name: serial port COM3'}]}
🔍 Testing COM4 → {'valid': False, 'problems': [{'character': 'COM4', 'reason': 'Reserved device name: serial port COM4'}]}
🔍 Testing COM5 → {'valid': False, 'problems': [

ok

----------------------------------------------------------------------
Ran 1 test in 3.743s

OK


🔍 Testing LPT9 → {'valid': False, 'problems': [{'character': 'LPT9', 'reason': 'Reserved device name: parallel port LPT9'}]}
